# Ozone vs airmass

- Goal check if files are filled correctly

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2024/11/17
- last update : 2024/11/17


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os,sys
import pickle
# to enlarge the sizes
params = {'legend.fontsize': 'large',
          'figure.figsize': (10, 6),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Imports

In [ ]:
import getObsAtmo as obs

In [ ]:
OBS_tag = 'LSST'

In [ ]:
obs.__path__

In [ ]:
from getObsAtmo.getObsAtmo import ObsAtmo,Dict_Of_sitesPressures,getObsSiteDataFrame,_getPackageDir

In [ ]:

__all__ = ['Dict_Of_sitesAltitudes',
           'Dict_Of_sitesPressures',
           'Dict_Of_sitesAliases',
           'file_data_dict',
           '_getPackageDir',
           'get_obssite_keys',
           'sanitizeString',
           'validateObsName', 
           'is_obssite', 
           'ObsAtmo','ObsAtmoPressure','ObsAtmoGrid']


# preselected sites
Dict_Of_sitesAltitudes = {'LSST': 2.663,
                          'CTIO': 2.207,
                          'OHP': 0.65,
                          'PDM': 2.8905,
                          'OMK': 4.205,
                          'OSL': 0.000,
                          }
# pressure calculated by libradtran
Dict_Of_sitesPressures = {'LSST': 731.50433,
                          'CTIO': 774.6052,
                          'OHP': 937.22595,
                          'PDM': 710.90637,
                          'OMK': 600.17224,
                          'OSL': 1013.000,
                          }

Dict_Of_sitesAliases = {'LSST': ['Rubin','Rubin Observatory','Auxtel'],
                        'CTIO': ["Cerro Tololo"],
                        'OHP': ["Observatoire de Haute Provence"],
                        'PDM': ["Pic du Midi","Observatoire du Pic du Midi"],
                        'OMK': ["Mauna Kea","Mauna Kea Observatory"],
                        'OSL': ["Sea Level","Sea Level Observatory"]
                        }

file_data_dict = {
    "info": "atmospherictransparencygrid_params.pickle",
    "data_rayleigh": "atmospherictransparencygrid_rayleigh.npy",
    "data_o2abs": "atmospherictransparencygrid_O2abs.npy",
    "data_pwvabs": "atmospherictransparencygrid_PWVabs.npy",
    "data_ozabs": "atmospherictransparencygrid_OZabs.npy",
}

In [ ]:
# construct the path of input data file
path = os.path.join(_getPackageDir(), '../obsatmo_data')
info_params = {}

# load all data files (training and test)
filename = os.path.join(path, OBS_tag + "_" + file_data_dict["info"])
with open(filename, 'rb') as f:
    info_params = pickle.load(f)

data_rayleigh = np.load(os.path.join(path, OBS_tag + "_" + file_data_dict["data_rayleigh"]))
data_O2abs = np.load(os.path.join(path, OBS_tag + "_" + file_data_dict["data_o2abs"]))
data_PWVabs = np.load(os.path.join(path, OBS_tag + "_" + file_data_dict["data_pwvabs"]))
data_OZabs = np.load(os.path.join(path, OBS_tag + "_" + file_data_dict["data_ozabs"]))

In [ ]:
# setup training dataset (those used for interpolation)
WLMIN = info_params["WLMIN"]
WLMAX = info_params["WLMAX"]
WLBIN = info_params["WLBIN"]
NWLBIN = info_params['NWLBIN']
WL     = info_params['WL']
OBS = info_params['OBS']

AIRMASSMIN = info_params['AIRMASSMIN']
AIRMASSMAX = info_params['AIRMASSMAX']
NAIRMASS = info_params['NAIRMASS']
DAIRMASS = info_params['DAIRMASS']
AIRMASS = info_params['AIRMASS']

PWVMIN = info_params['PWVMIN']
PWVMAX = info_params['PWVMAX']
NPWV = info_params['NPWV']
DPWV = info_params['DPWV']
PWV = info_params['PWV']

OZMIN = info_params['OZMIN']
OZMAX = info_params['OZMAX']
NOZ = info_params['NOZ']
DOZ = info_params['DOZ']
OZ = info_params['OZ']


In [ ]:
data_PWVabs.shape

In [ ]:
data_PWVabs.min()

# Check information

In [ ]:
Dict_Of_sitesPressures

### Choose Rubin-LSST observation sites but other sites may be chosen

In [ ]:
obs_str = 'LSST'

In [ ]:
emul =  ObsAtmo(obs_str=obs_str)

# Typical transmission plot

- show a typical transmission plot

In [ ]:
emul.plot_transmission()

# Setting atmospheric parameters

In [ ]:
WL = emul.GetWL()
airmasses = emul.AIRMASS
NAM = len(airmasses)
pwvs = emul.PWV
NPWV = len(pwvs)
ozs = emul.OZ
NOZ = len(ozs)

In [ ]:
hls_colors = sns.color_palette("hls", NAM)
sns.palplot(hls_colors)

## Check emulator with Ozone absorption

- expect no difference

In [ ]:
hls_colors = sns.color_palette("hls", NOZ)
sns.palplot(hls_colors)

In [ ]:
am =1

fig, ax = plt.subplots()
for idx,oz in enumerate(ozs):

    transm = emul.GetOZabsTransparencyArray(WL,am,oz)

    
    if idx == 0:
        ax.plot(WL,transm,"-",color=hls_colors[idx],label="")
    else:
        ax.plot(WL,transm,"-",color=hls_colors[idx])
     
title = f" ObsAtmo : Ozone absorption for different pressure, am = {am:.1f}"
ax.set_title(title)
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("transmission")
ax.set_xlim(300.,800.)
ax.set_ylim(0.9,1.0)
# place a text box in upper left in axes coords
#ax.text(0.5, 0.05, textboxstr, transform=ax.transAxes, fontsize=14,verticalalignment='bottom', bbox=textboxprops)
ax.grid()
ax.legend()
plt.show()

## Dependence with airmass

In [ ]:
WLtest = np.arange(400,850,50)
NWLtest = len(WLtest)
hls_colors = sns.color_palette("Spectral_r", NWLtest)
sns.palplot(hls_colors)

In [ ]:
airmasses

In [ ]:
oz0=300.
transmarray1 = np.zeros((NAM,NWLtest))
for idx,airmass in enumerate(airmasses):
    transmarray1[idx,:]  = emul.GetOZabsTransparencyArray(WLtest,airmass,oz0)
for idx,airmass in enumerate(airmasses):
     transmarray1[idx,:] =  transmarray1[idx,:]/transmarray1[0,:]

In [ ]:
oz0=600
transmarray2 = np.zeros((NAM,NWLtest))
for idx,airmass in enumerate(airmasses):
    transmarray2[idx,:]  = emul.GetOZabsTransparencyArray(WLtest,airmass,oz0)
for idx,airmass in enumerate(airmasses):
     transmarray2[idx,:] =  transmarray2[idx,:]/transmarray2[0,:]

In [ ]:
fig, ax = plt.subplots()
for idx,wl in enumerate(WLtest):
    color = hls_colors[idx]
    label = "$\\lambda$ = "+f"{wl:.0f} nm"
    ax.plot(airmasses,transmarray2[:,idx],'-o',color=color,lw=5,label=label)
    ax.plot(airmasses,transmarray1[:,idx],':o',color=color,lw=5)
ax.legend()
ax.grid()
ax.set_title(f"Ozone attenuation with airmass for oz = 300 and 600 DU")
ax.set_ylabel("Ozone relative attenuation")
ax.set_xlabel("airmass")
ax.set_ylim(0.8,1.01)
ax.set_xscale("log")
ax.set_yscale("log")

## Fit

In [ ]:
xfit = np.linspace(np.log(airmasses).min(),np.log(airmasses).max(),50)
fig, ax = plt.subplots()
for idx,wl in enumerate(WLtest):
    color = hls_colors[idx]
    x = np.log(airmasses)
    y = np.log(transmarray1[:,idx])
    z = np.polyfit(x[2:], y[2:], deg=1)
    p = np.poly1d(z)
    gamma = z[0]
    const = z[1]
    label = "$\\lambda$ = "+f"{wl:.0f} nm"+ f"fit : slope = {gamma:.3f} const = {const:.3f}"
    ax.plot(x,y,'-o',color=color,lw=5,label=label)
    ax.plot(xfit,p(xfit),":k")
   
ax.legend()
ax.grid()
ax.set_title(f"Ozone attenuation with airmass for oz = 300 DU")
ax.set_ylabel("Ozone relative attenuation")
ax.set_xlabel("airmass")
#ax.set_ylim(0.8,1.01)
#ax.set_xlim(1,airmasses.max())
#ax.set_xscale("log")
#ax.set_yscale("log")

In [ ]:
xfit = np.linspace(np.log(airmasses).min(),np.log(airmasses).max(),50)
fig, ax = plt.subplots()
for idx,wl in enumerate(WLtest):
    color = hls_colors[idx]
    x = np.log(airmasses)
    y = np.log(transmarray2[:,idx])
    z = np.polyfit(x[2:], y[2:], deg=1)
    p = np.poly1d(z)
    gamma = z[0]
    const = z[1]
    label = "$\\lambda$ = "+f"{wl:.0f} nm"+ f"fit : slope = {gamma:.3f} const = {const:.3f}"
    ax.plot(x,y,'-o',color=color,lw=5,label=label)
    ax.plot(xfit,p(xfit),":k")
   
ax.legend()
ax.grid()
ax.set_title(f"Ozone attenuation with airmass for oz = 600 DU")
ax.set_ylabel("Ozone relative attenuation")
ax.set_xlabel("airmass")
#ax.set_ylim(0.8,1.01)
#ax.set_xlim(1,airmasses.max())
#ax.set_xscale("log")
#ax.set_yscale("log")